# Project-15
## Nội dung :
Có N lớp 1,2,..., N cần được xếp thời khóa biểu. Mỗi lớp i có t(i) là số tiết và g(i) là giáo viên đã được phân công dạy lớp đó và s(i) là số sinh viên của lớp 
Có M phòng học 1, 2, ..., M, trong đó c(i) là số chỗ ngồi của phòng i  Trong tuần có 5 ngày (từ thứ 2 đến thứ 5), mỗi ngày chia thành 12 tiết (6 tiết sáng và 6 tiết chiều).  
Hãy lập thời khóa biểu (xác định ngày, tiết và phòng gán cho mỗi lớp)  
Hai lớp có chung giáo viên thì phải xếp thời khóa biểu tách rời nhau  
Số sinh viên trong mỗi lớp phải nhỏ hơn hoặc bằng số chỗ ngồi của phòng học

## Dữ liệu đầu vào :
File data: "data.txt"
> Dòng 1: ghi N và M <br>
> Dòng i+1 (i = 1,…, N): ghi t(i), g(i) và s(i)  
> Dòng N+2: ghi c(1), c(2), …, c(M)

## Đặt biến
n Lớp: l từ 0,1,2...n-1
- T[l] Số tiết - tuple
-	G[l] Giáo viên dạy lớp đó - tuple
-	S[l] số học sinh - tuple <br>

Từ G tạo ra 1 list D_G có index là giáo viên-1, element là list các lớp giáo viên đó dạy.


m Phòng: p từ 0,1,2,..m-1
-	C - số chỗ trong tất cả các phòng
-	T’ - Số tiết trong 10 buổi của tất cả các phòng


Phòng số p có:
- C[p] Số chỗ trong phòng
- T[p][b] Số tiết còn lại trong buổi thứ b
VD:
2 phòng 1,2
3 buổi 1,2,3
C = [40,30]
T = [[6,4,5],[3,2,5]]
=> Phòng số 1 có 40 chỗ và còn trống 4 tiết vào buổi thứ 2

Biến lựa chọn - dict
(Lớp,phòng,buổi,tiết) = 0 hoặc 1 - (6*10*n*m) biến
Lớp 5 xếp vào phòng 1 trong buổi 2 tiết 1, Lớp 6 không xếp vào phòng 1 trong buổi 2 tiết 2 - { (5,1,2,1):1, (6,1,2,2):0 }
### Input 

In [2]:
from ortools.sat.python import cp_model
def input(FileName):
    f = open(FileName,'r')
    (n, m) =  (int(i) for i in f.readline().split())
    T,S,G,D_G,C = [],[],[],[],[]
    for c in range(n):
        [so_tiet, gv, so_hs] = (int(i) for i in f.readline().split())
        T.append(so_tiet)
        S.append(so_hs)
        G.append(gv)
        if len(D_G)<gv:
            D_G.append([c])
        else:
            D_G[gv-1].append(c)
    C = [int(i) for i in f.readline().split()]
    return (n,m,T,S,G,D_G,C)

FileName= 'data.txt'
n,m,T,S,G,D_G,C = input(FileName)
so_buoi,so_tiet= 10,6
all_gv = range(len(D_G))
all_p = range(m)
all_b = range(so_buoi)
all_t = range(so_tiet)
all_l = range(n)

## Constraint 1
Nếu lớp được xếp vào phòng, số học sinh phải nhỏ hơn số chỗ ngồi <br>
>$\forall l \in \text{ all_l}, \forall p \in \text{ all_p}, \forall b \in \text{all_b}, \forall t \in \text{all_t}:$ <br>
>$\displaystyle \text{If } S[l] > C[p]: lc(l,p,b,t) = 0 $


## Constraint 2
Trong cùng 1 thời gian:
 - 1 phòng chỉ chứa nhiều nhất 1 lớp <br>
   >$\displaystyle \sum_{l \in \text{ all_l}} lc(l,p,b,t) \le 1, \forall p \in \text{ all_p}, \forall b \in \text{all_b}, \forall t \in \text{all_t}$

- 1 giáo viên không được dạy 2 lớp khác nhau <br>
>   $\displaystyle \sum_{l \in \text{ D_G[g]}} \text{ }\sum_{p \in \text{ all_p}} lc(l,p,b,t) \le 1, \forall g \in \text{all_gv},\forall b \in \text{all_b}, \forall t \in \text{all_t}$


## Constraint 3
Mỗi lớp:
- Học đủ số tiết<br>
>$\displaystyle \sum_{b \in \text{ all_b}} \text{ }\sum_{p \in \text{ all_p}} \text{ }\sum_{t \in \text{ all_t}} lc(l,p,b,t) = T[l], \forall l \in \text{all_l}$
-  Chỉ học trong 1 phòng, 1 buổi duy nhất và các tiết phải liền nhau trong buổi đó<br>
>$\forall l \in \text{ all_l}, \forall p \in \text{ all_p}, \forall b \in \text{all_b}:$ <br>
>$\displaystyle \text{If} \sum_{t \in \text{ all_t}} lc(l,p,b,t) \neq 0: \exists i \in [0 \text{ ; } 6-T[l]]: \sum_{k=i} ^{i+T[l]-1} lc(l,p,b,k) = T[l] $

>$\displaystyle \prod_{b \in \text{ all_b}} \text{ } \prod_{p \in \text{ all_p}} (T[l]-\sum_{t \in \text{ all_t}} lc(l,p,b,t)) = 0 , \forall l \in \text{all_l}$